In [2]:
import pandas as pd
from itertools import groupby


data = pd.read_csv('res.csv')
data['length'] = ''
data['label'] = ''
data['first_seen'] = ''

# 去除掉缺失值
data = data.dropna(subset=['opcode'])
data = data.reset_index(drop=True)

# 对opcode序列进行去重处理
def handle(l):
    x_res = [x[0] for x in groupby(l)]
    return x_res
data['opcode'] = data['opcode'].apply(lambda x: x.split())
data['opcode'] = data['opcode'].apply(lambda x: handle(x))
for idx in range(data.shape[0]):
    data.at[idx, 'length'] = len(data.at[idx, 'opcode'])
data['length'] = data['length'].astype(int)

# 将opcode列表转换为字符串
data['opcode'] = data['opcode'].apply(lambda x: ' '.join(x))

# 去除掉只包含一个opocde的样本
data = data[data['length'].apply(lambda x: x > 1)]
data = data.reset_index(drop=True)

In [3]:
# 对样本opcode的长度进行统计

data['length'].describe()

count     12402.000000
mean       5652.612804
std       20251.140030
min           2.000000
25%         202.000000
50%        1105.500000
75%        2849.000000
max      780386.000000
Name: length, dtype: float64

In [4]:
# 统计总共有多少个不同的opcode

from collections import Counter

opcode_str = ''
for i in range(data.shape[0]):
    opcode_str += data.at[i, 'opcode']
opcode_list = opcode_str.split()
info = Counter(opcode_list)
len(info)

963

In [5]:
data.head()

,name,path,opcode,length,label,first_seen
0,00033a9606f5fe9ed77b994f6b8e42901f5319783d3850...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\000...,mov cmp mov push retn,5,,
1,004804cd6b95e1a5aa09d6c893ac51ace90e1c9b16cb2c...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\004...,retn mov test jz push mov sub mov call mov sub...,2989,,
2,005e0b9944f5f207b45e66493c378a6afc6bffba2030e5...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\005...,push mov sub cmp jz cmp mov jnz or mov push ca...,5920,,
3,006c65515ea0c4998bee697c3962a3c4fc225afd5f934f...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\006...,nop pusha nop mov nop push nop pop nop cmp nop...,120,,
4,007713edd1589b07bb902b7481dec4bc92bd3dc37b30d8...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\007...,push call cmp jz retn mov push call mov pop mo...,77,,


In [8]:
# 加载样本标签csv文件

data_label = pd.read_csv('../../csv/dataset.csv')
data_label.head()

,name,first_seen,scan_date,size,trojan,virus,worm,backdoor,label
0,../data/2017-10-20/1d609fff68c89667ba51cb707cc...,2017-10-18,2017-10-20,3555328,19,0,3,21,backdoor
1,../data/2017-10-20/bc473631b56f37e7cc96b9c9a79...,2017-09-20,2017-10-20,1696353,17,0,3,20,backdoor
2,../data/2017-10-20/e2a802bd3b29472a99f2601e3e1...,2017-10-20,2017-10-20,1774510,17,0,4,19,backdoor
3,../data/2017-10-20/047921d3f50092a4ec4794f954f...,2017-09-20,2017-10-20,3577275,18,0,4,21,backdoor
4,../data/2017-10-20/9a1bf0da31927f7989e6305231f...,2017-10-20,2017-10-20,2042301,17,0,3,20,backdoor


In [9]:
# 为opcode序列打上标签
from tqdm import tqdm

for idx in tqdm(range(data.shape[0])):
    op_name = data.at[idx, 'name']
    for idx_label in range(data_label.shape[0]):
        if op_name in data_label.at[idx_label, 'name']:
            data.at[idx, 'label'] = data_label.at[idx_label, 'label']
            data.at[idx, 'first_seen'] = data_label.at[idx_label, 'first_seen']
            break

100%|████████████████████████████████████████████████████████████████████████████| 12402/12402 [07:04<00:00, 29.24it/s]


In [10]:
data.head()

,name,path,opcode,length,label,first_seen
0,00033a9606f5fe9ed77b994f6b8e42901f5319783d3850...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\000...,mov cmp mov push retn,5,trojan,2017-10-18
1,004804cd6b95e1a5aa09d6c893ac51ace90e1c9b16cb2c...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\004...,retn mov test jz push mov sub mov call mov sub...,2989,virus,2017-09-20
2,005e0b9944f5f207b45e66493c378a6afc6bffba2030e5...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\005...,push mov sub cmp jz cmp mov jnz or mov push ca...,5920,trojan,2017-09-20
3,006c65515ea0c4998bee697c3962a3c4fc225afd5f934f...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\006...,nop pusha nop mov nop push nop pop nop cmp nop...,120,trojan,2017-10-20
4,007713edd1589b07bb902b7481dec4bc92bd3dc37b30d8...,C:\Users\bitcjx\Desktop\dataset\2017-10-20\007...,push call cmp jz retn mov push call mov pop mo...,77,trojan,2017-10-20


In [11]:
# 删除掉path列并将最后的结果保存为csv文件

data.drop('path', axis=1, inplace=True)
data.to_csv('res_handle.csv', index=0)

In [8]:
data = pd.read_csv('res_handle.csv')
name = data.at[0, 'name']

In [9]:
op_string = data[data.name == name].opcode

In [13]:
op_string[0].split()

['mov', 'cmp', 'mov', 'push', 'retn']